# Libraries

In [1]:
%matplotlib inline
import cv2
import os

import numpy as np
import mediapipe as mp

from mp_support import mediapipe_detection, draw_landmarks, draw_styled_landmarks, extract_keypoints, extract_coordinates, prob_viz

# Import mediapipe model for finger and pose detection

In [2]:
mp_holistic = mp.solutions.holistic # holistic model
mp_drawing = mp.solutions.drawing_utils # drawing utilities

# Folder creation

Create folders for
1. Label folders for new actions
2. Sub-label folders

In [8]:
# base directory
directory_path = './augmented_data'
# current directory
c_dir = os.getcwd()

actions = np.array(sorted([folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))])) # sorted to follow folder arrangement
# actions = np.array(['afternoon', 'house', 'again'])

In [9]:
actions

array(['afternoon', 'again', 'base', 'door', 'hello', 'house', 'how',
       'open', 'see', 'sorry', 'why'], dtype='<U9')

# Keypoint extraction
Here we extract the keypoints of the data (videos) by looping through each set of videos in each action folder, then writing the corresponding keypoints. 

This will be used later for model training.

In [10]:
video_count = 0 # required for processing later
for action in actions:
    counter = 0 # to count video/extracted file
    if not os.path.isdir(os.path.join(c_dir, 'labels_new', action)):
        os.mkdir(os.path.join(c_dir, 'labels_new', action))
    for video in os.listdir(os.path.join(directory_path, action)): # going through each converted video file in the action
        counter += 1
        video_count += 1
        filepath = os.path.join(os.path.join(c_dir, 'augmented_data', action, video))

        # NOTE: for each video, save keypoints in new subfolder
        # create subfolder if it does not exist
        subfolder = str(action) + '_' + str(counter)
        if not os.path.isdir(os.path.join(c_dir, 'labels_new', action, subfolder)):
            os.mkdir(os.path.join(c_dir, 'labels_new', action, subfolder))
        # set new savepath
        savepath = os.path.join(c_dir, 'labels_new', action, subfolder)
        print(f'Currently reading video {video_count}: {filepath}')
        extract_coordinates(filepath, savepath)


Currently reading video 1: d:\GitHub\SSLrecognition\working\augmented_data\afternoon\Afternoon_10_001.mp4


c:\Users\wei-z\anaconda3\envs\slproj\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Currently reading video 2: d:\GitHub\SSLrecognition\working\augmented_data\afternoon\Afternoon_10_002.mp4
Currently reading video 3: d:\GitHub\SSLrecognition\working\augmented_data\afternoon\Afternoon_10_003.mp4
Currently reading video 4: d:\GitHub\SSLrecognition\working\augmented_data\afternoon\Afternoon_10_004.mp4
Currently reading video 5: d:\GitHub\SSLrecognition\working\augmented_data\afternoon\Afternoon_10_005.mp4
Currently reading video 6: d:\GitHub\SSLrecognition\working\augmented_data\afternoon\Afternoon_10_006.mp4
Currently reading video 7: d:\GitHub\SSLrecognition\working\augmented_data\afternoon\Afternoon_10_007.mp4
Currently reading video 8: d:\GitHub\SSLrecognition\working\augmented_data\afternoon\Afternoon_10_008.mp4
Currently reading video 9: d:\GitHub\SSLrecognition\working\augmented_data\afternoon\Afternoon_10_009.mp4
Currently reading video 10: d:\GitHub\SSLrecognition\working\augmented_data\afternoon\Afternoon_10_010.mp4
Currently reading video 11: d:\GitHub\SSLreco

# Mediapipe to dataframe Conversion

In [6]:
import pandas as pd

df = pd.read_parquet('635217.parquet')
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

# Display the entire DataFrame
print(df)

      frame            row_id        type  landmark_index         x         y  \
0        22         22-face-0        face               0  0.438251  0.449453   
1        22         22-face-1        face               1  0.414527  0.404880   
2        22         22-face-2        face               2  0.423745  0.420681   
3        22         22-face-3        face               3  0.402349  0.372041   
4        22         22-face-4        face               4  0.411857  0.393013   
5        22         22-face-5        face               5  0.411587  0.379338   
6        22         22-face-6        face               6  0.413674  0.347887   
7        22         22-face-7        face               7  0.340341  0.361587   
8        22         22-face-8        face               8  0.411991  0.323421   
9        22         22-face-9        face               9  0.409135  0.308535   
10       22        22-face-10        face              10  0.401925  0.252454   
11       22        22-face-1

In [ ]:
import os
import pandas as pd
import numpy as np

directory_path = "./labels_final"
# actions = np.array(["all"])

# for action in folder
for action in os.listdir(directory_path):
    action_path = os.path.join(directory_path, action)
    if os.path.isdir(action_path):
        # for video folder
        for video in os.listdir(action_path):
            video_path = os.path.join(action_path, video)
            # create empty data frame
            df = pd.DataFrame(columns=['frame', 'row_id', 'type', 'landmark_index', 'x', 'y', 'z'])
            frame_counter = 1 
            # for video
            for frame in os.listdir(video_path):
                frame_path = os.path.join(video_path, frame)
                npy_data = np.load(frame_path)
                # scan through npy file and add data to df in batches of 3
                for j in range(0, 99, 3):
                    # frame = i
                    # read npy
                    new_row = {'frame': frame_counter, 'row_id': f"{frame_counter}-pose-{j}", 'type': 'pose', 'landmark_index': j, 'x': npy_data[j], 'y': npy_data[j+1], 'z': npy_data[j+2]}
                    pd.concat([df, pd.DataFrame(new_row, index = [0])], ignore_index = True)
                for j in range(99, 162, 3):
                    new_row = {'frame': frame_counter, 'row_id': f"{frame_counter}-left_hand-{j}", 'type': 'left_hand', 'landmark_index': j, 'x': npy_data[j], 'y': npy_data[j+1], 'z': npy_data[j+2]}
                    pd.concat([df, pd.DataFrame(new_row, index = [0])], ignore_index = True)
                for j in range(162, 225, 3):
                    new_row = {'frame': frame_counter, 'row_id': f"{frame_counter}-right_hand-{j}", 'type': 'right_hand', 'landmark_index': j, 'x': npy_data[j], 'y': npy_data[j+1], 'z': npy_data[j+2]}
                    pd.concat([df, pd.DataFrame(new_row, index = [0])], ignore_index = True)
            # check if is dir, if not, make dir
            if not os.path.exists(f"./labels_parquet/{action}"):
                # If it doesn't exist, create it
                os.makedirs(f"./labels_parquet/{action}")

            df.to_parquet(f"./labels_parquet/{action}/{video}.parquet")

In [27]:
df

,frame,row_id,type,landmark_index,x,y,z
0,1,1-pose-0,pose,0,0.444283,0.319614,-2.864992


# Kaggle Conversion

In [3]:
from zipfile import ZipFile

with ZipFile("./averaged_np_labels(3).zip", 'r') as zObject: 
    zObject.extractall(path="./") 

In [5]:
import os
import shutil

folder_path = './averaged_np_labels/'
parent_files = os.listdir(os.path.join(folder_path))
write_path = './labels_final/'

for parent_file in parent_files:
    parent_path = os.path.join(folder_path, parent_file)
    export_path = os.path.join(write_path, parent_file)
    os.mkdir(export_path)
    print(f'Created new directory: {export_path}')
    for i in range(1, len(os.listdir(os.path.join(folder_path, parent_file)))+1):
        new_subfolder = f'{parent_file}_{i}'
        os.mkdir(os.path.join(export_path, new_subfolder))
        print(f'Created new subdirectory: {new_subfolder}')

        source = os.path.join(parent_path, f'video{i}')
        destination = os.path.join(os.path.join(export_path, new_subfolder))

        sourcefolder = os.listdir(os.path.join(parent_path, f'video{i}'))
        for file in sourcefolder:
            file_to_copy = os.path.join(os.path.join(parent_path, f'video{i}'), file)
            shutil.copy(file_to_copy, destination)

Created new directory: ./labels_final/please
Created new subdirectory: please_1
Created new subdirectory: please_2
Created new subdirectory: please_3
Created new subdirectory: please_4
Created new subdirectory: please_5
Created new subdirectory: please_6
Created new subdirectory: please_7
Created new subdirectory: please_8
Created new subdirectory: please_9
Created new subdirectory: please_10
Created new subdirectory: please_11
Created new subdirectory: please_12
Created new subdirectory: please_13
Created new subdirectory: please_14
Created new subdirectory: please_15
Created new subdirectory: please_16
Created new subdirectory: please_17
Created new subdirectory: please_18
Created new subdirectory: please_19
Created new subdirectory: please_20
Created new subdirectory: please_21
Created new subdirectory: please_22
Created new subdirectory: please_23
Created new subdirectory: please_24
Created new subdirectory: please_25
Created new subdirectory: please_26
Created new subdirectory: pl

# Live Test

In [17]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
actions = np.array(sorted([folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))]))

In [13]:
import keras
import cv2

In [18]:
# Load pre-trained Keras model
lstm_model = keras.models.load_model('lstm_model.h5')

sequence = []
sentence = []
predictions = []
threshold = 0.3

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 267:
            res = lstm_model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti